# MODEL PREDICTION

We will predict the results of prediction on the test images provided in the competition using the best models generated from each of the following approaches 

Approaches followed for model creation
1. Creating the Self designned CNN from scratch 
2. Using the Vgg16 architecture as a base and fine tuning the last layer

In [ ]:
#Importing all the requied libraries , which I used in this Project. 
import os
import json
from keras.models import load_model
import pandas as pd
import pickle
import numpy as np
import shutil

from keras.preprocessing import image                  
from tqdm.notebook import tqdm
from PIL import ImageFile                            


# Defining the train,test and model directories

* We will create the directories for train,test and model training paths if not present

In [ ]:
# That's are my model path directories
BASE_MODEL_PATH = os.path.join(os.getcwd(),"model")
TEST_DIR = os.path.join(os.getcwd(),"csv_files","test.csv")
PREDICT_DIR = os.path.join(os.getcwd(),"pred_dir")
PICKLE_DIR = os.path.join(os.getcwd(),"pickle_files")
JSON_DIR = os.path.join(os.getcwd(),"json_files")

In [ ]:
# Using condition to confirm that have all above defined directories are their or not . 
if not os.path.exists(PREDICT_DIR):
    os.makedirs(PREDICT_DIR)
else:
    shutil.rmtree(PREDICT_DIR)
    os.makedirs(PREDICT_DIR)
if not os.path.exists(JSON_DIR):
    os.makedirs(JSON_DIR)

# Prediction using-->Self Trained Model (CNN Scratch)

In [ ]:
# Generate path with self trained model weight
BEST_MODEL = os.path.join(BASE_MODEL_PATH,"self_trained","distracted-23-0.99.hdf5")
model = load_model(BEST_MODEL)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       32896     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 256)       131328    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 512)       5

In [ ]:
# Read the all test images 
data_test = pd.read_csv(os.path.join(TEST_DIR))
#testing on the only 10000 images as loading the all test images requires ram>8gb
data_test = data_test[:10000] 
data_test.info()
# data_test.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   FileName   100 non-null    object
 1   ClassName  100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [ ]:
# Converts a Python object hierarchy into a byte stream written to a buffer or to a disk file.
with open(os.path.join(PICKLE_DIR,"labels_list.pkl"),"rb") as handle:
    labels_id = pickle.load(handle)
print(labels_id)
# labels_id = {'c1': 0, 'c6': 1, 'c5': 2, 'c8': 3, 'c3': 4, 'c0': 5, 'c7': 6, 'c2': 7, 'c4': 8, 'c9': 9}

{'c5': 0, 'c8': 1, 'c1': 2, 'c0': 3, 'c4': 4, 'c3': 5, 'c6': 6, 'c9': 7, 'c2': 8, 'c7': 9}


# Labeling & Testing
                    Using library function ImageFile to make file or directory situable for pictures. 

In [ ]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(128, 128))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

ImageFile.LOAD_TRUNCATED_IMAGES = True  
test_tensors = paths_to_tensor(data_test.iloc[:,0]).astype('float32')/255 - 0.5

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# predicting model at both train and valid with single single animated progress bar 
ypred_test = model.predict(test_tensors,verbose=1)
ypred_class = np.argmax(ypred_test,axis=1)

id_labels = dict()
for class_name,idx in labels_id.items():
    id_labels[idx] = class_name
print(id_labels) 

for i in range(data_test.shape[0]):
    data_test.iloc[i,1] = id_labels[ypred_class[i]]

In [ ]:
#to create a human readable and understandable class_name 
class_name = dict()
class_name["c0"] = "SAFE_DRIVING"
class_name["c1"] = "TEXTING_RIGHT"
class_name["c2"] = "TALKING_PHONE_RIGHT"
class_name["c3"] = "TEXTING_LEFT"
class_name["c4"] = "TALKING_PHONE_LEFT"
class_name["c5"] = "OPERATING_RADIO"
class_name["c6"] = "DRINKING"
class_name["c7"] = "REACHING_BEHIND"
class_name["c8"] = "HAIR_AND_MAKEUP"
class_name["c9"] = "TALKING_TO_PASSENGER"


with open(os.path.join(JSON_DIR,'class_name_map.json'),'w') as secret_input:
    json.dump(class_name,secret_input,indent=4,sort_keys=True)


In [ ]:
# creating the prediction results for the image classification and shifting the predicted images to another folder
#with renamed filename having the class name predicted for that image using model
with open(os.path.join(JSON_DIR,'class_name_map.json')) as secret_input:
    info = json.load(secret_input)


for i in range(data_test.shape[0]):
    new_name = data_test.iloc[i,0].split("/")[-1].split(".")[0]+"_"+info[data_test.iloc[i,1]]+".jpg"
    shutil.copy(data_test.iloc[i,0],os.path.join(PREDICT_DIR,new_name))
    
#saving the model predicted results into a csv file
data_test.to_csv(os.path.join(os.getcwd(),"csv_files","short_test_result.csv"),index=False)